In [1]:
%mkdir ../data

In [2]:
# http://jmcauley.ucsd.edu/data/amazon/
!wget -O ../data/amazon_data.tar.gz http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz
!gunzip ../data/amazon_data.tar.gz
!mv ../data/amazon_data.tar ../data/amazon_data.json 
# !tar -zvxf ../data/amazon_data.tar.gz -C ../data

--2021-05-18 20:09:34--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36119369 (34M) [application/x-gzip]
Saving to: ‘../data/amazon_data.tar.gz’

../data/amazon_data 100%[===================>]  34.45M  12.1MB/s    in 2.8s    

2021-05-18 20:09:37 (12.1 MB/s) - ‘../data/amazon_data.tar.gz’ saved [36119369/36119369]



In [3]:
!ls ../data

amazon_data.json


In [4]:
import pandas as pd

df = pd.read_json('../data/amazon_data.json', lines=True)

In [5]:
# del(df['reviewerID'])
del(df['reviewerName'])
del(df['helpful'])
del(df['unixReviewTime'])
del(df['reviewTime'])
del(df['asin'])
del(df['reviewerID'])

In [6]:
df.head()

,reviewText,overall,summary
0,I purchased the Trilogy with hoping my two cat...,3,Nice Distraction for my cats for about 15 minutes
1,There are usually one or more of my cats watch...,5,Entertaining for my cats
2,I bought the triliogy and have tested out all ...,4,Entertaining
3,My female kitty could care less about these vi...,4,Happy to have them
4,"If I had gotten just volume two, I would have ...",3,You really only need vol 2


In [7]:
df.count()

reviewText    157836
overall       157836
summary       157836
dtype: int64

In [8]:
import sagemaker

session = sagemaker.Session()

s3_prefix = 'review_data'

reviews_dir = '../data/reviews_raw.pickle'

df.to_pickle(reviews_dir)

input_data = session.upload_data(reviews_dir,'sagemaker-capstone-project', key_prefix=s3_prefix)

# Data Preprocessing

In [8]:
from string import punctuation

In [9]:
df['reviewText'] = df['reviewText'].str.lower()
df['summary'] = df['summary'].str.lower()
df['summary_'] = df['summary'].apply(lambda c: c if c not in punctuation else '')
df['summary_'].str.strip()

0         nice distraction for my cats for about 15 minutes
1                                  entertaining for my cats
2                                              entertaining
3                                        happy to have them
4                                you really only need vol 2
                                ...                        
157831                                           perfection
157832          nice solid feel, my little boy is thrilled!
157833                  good leash for small non-darty pets
157834        quality build, smooth retraction, great leash
157835    nice leash for 20+ pounds, occasional problems...
Name: summary_, Length: 157836, dtype: object

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [11]:
import re
from bs4 import BeautifulSoup

In [12]:
nltk.download("stopwords", quiet=True)

True

In [13]:
def preprocessing(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    return words

In [14]:
df['summary__'] = df['summary_'].apply(lambda c: preprocessing(c))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "http://www.amazon.com/gp/product/b00a303gl2/ref=cm_cr_ryp_prd_ttl_sol_1" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


In [15]:
df['summary__'].iloc[10]

['puppi', 'wont', 'eat', 'food']

In [16]:
df.head()

,reviewText,overall,summary,summary_,summary__
0,i purchased the trilogy with hoping my two cat...,3,nice distraction for my cats for about 15 minutes,nice distraction for my cats for about 15 minutes,"[nice, distract, cat, 15, minut]"
1,there are usually one or more of my cats watch...,5,entertaining for my cats,entertaining for my cats,"[entertain, cat]"
2,i bought the triliogy and have tested out all ...,4,entertaining,entertaining,[entertain]
3,my female kitty could care less about these vi...,4,happy to have them,happy to have them,[happi]
4,"if i had gotten just volume two, i would have ...",3,you really only need vol 2,you really only need vol 2,"[realli, need, vol, 2]"


In [18]:
# import sagemaker

# session = sagemaker.Session()

# s3_prefix = 'review_data'

# reviews_dir = '../data/reviews_preprocess.pickle'

# df.to_pickle(reviews_dir)

# input_data = session.upload_data(reviews_dir,'sagemaker-capstone-project', key_prefix=s3_prefix)

In [19]:
# import sagemaker

# session = sagemaker.Session()

# s3_prefix = 'review_data'

In [20]:
# session.download_data('../data','sagemaker-capstone-project' , key_prefix=s3_prefix)

## Create vocabulary

In [23]:
# df = None
# df = pd.read_pickle("../data/reviews_preprocess.pickle")

In [20]:
len(df)

157836

In [21]:
df.head()

,reviewText,overall,summary,summary_,summary__
0,i purchased the trilogy with hoping my two cat...,3,nice distraction for my cats for about 15 minutes,nice distraction for my cats for about 15 minutes,"[nice, distract, cat, 15, minut]"
1,there are usually one or more of my cats watch...,5,entertaining for my cats,entertaining for my cats,"[entertain, cat]"
2,i bought the triliogy and have tested out all ...,4,entertaining,entertaining,[entertain]
3,my female kitty could care less about these vi...,4,happy to have them,happy to have them,[happi]
4,"if i had gotten just volume two, i would have ...",3,you really only need vol 2,you really only need vol 2,"[realli, need, vol, 2]"


In [17]:
from collections import Counter

voc = []
for x in df['summary__']:
    for y in x:
        voc.append(y)

counter = Counter(voc)

In [18]:
vocab = sorted(counter, key=counter.get, reverse=True)

In [19]:
len(vocab)

12984

In [20]:
# Removing the words that only appear once
words = {k:v for k,v in counter.items() if v>1}

In [21]:
words = sorted(words, key=words.get, reverse=True)

In [22]:
vocab_to_int = {word: ii for ii, word in enumerate(words, 1)}

In [23]:
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [24]:
len(word2idx) # len(words) 7263

7263

## Save word dict

In [529]:
# import os
# data_dir = '../data/pytorch' # The folder we will use for storing data
# if not os.path.exists(data_dir): # Make sure that the folder exists
#     os.makedirs(data_dir)

In [530]:
# import pickle
# import os
# with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
#     pickle.dump(word2idx, f)

## Apply word to number

In [25]:
#Remove 0 len reviews
df=df[(df['summary__'].str.len() > 0)]

#pad with 0's
# df['summary__'] = df['summary_'].apply(lambda c: c.zfill(20))

#truncate long reviews
df=df[(df['summary__'].str.len() < 200)]


In [26]:
df.head()

,reviewText,overall,summary,summary_,summary__
0,i purchased the trilogy with hoping my two cat...,3,nice distraction for my cats for about 15 minutes,nice distraction for my cats for about 15 minutes,"[nice, distract, cat, 15, minut]"
1,there are usually one or more of my cats watch...,5,entertaining for my cats,entertaining for my cats,"[entertain, cat]"
2,i bought the triliogy and have tested out all ...,4,entertaining,entertaining,[entertain]
3,my female kitty could care less about these vi...,4,happy to have them,happy to have them,[happi]
4,"if i had gotten just volume two, i would have ...",3,you really only need vol 2,you really only need vol 2,"[realli, need, vol, 2]"


In [27]:
df.count() #157531

reviewText    157531
overall       157531
summary       157531
summary_      157531
summary__     157531
dtype: int64

In [28]:
# df[df.summary__.str.contains(r'[1234567890]')]
def W2I(sumary):
    w2i = []
    for y in sumary:
        w2i.append(word2idx[y] if y in word2idx else 0 )
    return w2i

In [29]:
def pad(w2i, pad):
    x = len(w2i)
    for i in range(pad - x) :
        w2i.insert(0, 0)
    return w2i

In [30]:
df['W2I'] = df['summary__'].apply(lambda c: W2I(c))

In [31]:
df['W2I'] = df['W2I'].apply(lambda c: pad(c, 20))

In [32]:
df.head()

,reviewText,overall,summary,summary_,summary__,W2I
0,i purchased the trilogy with hoping my two cat...,3,nice distraction for my cats for about 15 minutes,nice distraction for my cats for about 15 minutes,"[nice, distract, cat, 15, minut]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,there are usually one or more of my cats watch...,5,entertaining for my cats,entertaining for my cats,"[entertain, cat]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,i bought the triliogy and have tested out all ...,4,entertaining,entertaining,[entertain],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,my female kitty could care less about these vi...,4,happy to have them,happy to have them,[happi],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"if i had gotten just volume two, i would have ...",3,you really only need vol 2,you really only need vol 2,"[realli, need, vol, 2]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [33]:
# seq_length = 200

# ## test statements - do not change - ##
# assert len(df)==len(word2idx), "Your features should have as many rows as reviews."
assert len(df['W2I'][0] )==20, "Each feature row should contain seq_length values."

In [34]:
import numpy as np

def pad_overall(review):
    review_ranked = np.zeros(5)
    review_ranked[review-1] = 1
    return review_ranked

In [35]:
df['overall_'] = df['overall'].apply(lambda c:  pad_overall(c))

In [36]:
column = 'overall'
df['overall_'] = df[column] / df[column].abs().max()
  
# view normalized data
display(df)

,reviewText,overall,summary,summary_,summary__,W2I,overall_
0,i purchased the trilogy with hoping my two cat...,3,nice distraction for my cats for about 15 minutes,nice distraction for my cats for about 15 minutes,"[nice, distract, cat, 15, minut]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.6
1,there are usually one or more of my cats watch...,5,entertaining for my cats,entertaining for my cats,"[entertain, cat]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
2,i bought the triliogy and have tested out all ...,4,entertaining,entertaining,[entertain],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.8
3,my female kitty could care less about these vi...,4,happy to have them,happy to have them,[happi],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.8
4,"if i had gotten just volume two, i would have ...",3,you really only need vol 2,you really only need vol 2,"[realli, need, vol, 2]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.6
...,...,...,...,...,...,...,...
157831,the pet magasin retractable dog leash is the b...,5,perfection,perfection,[perfect],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
157832,i'm not the biggest fan of retractable leashes...,5,"nice solid feel, my little boy is thrilled!","nice solid feel, my little boy is thrilled!","[nice, solid, feel, littl, boy, thrill]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15,...",1.0
157833,i was provided a leash for my unbiased review....,4,good leash for small non-darty pets,good leash for small non-darty pets,"[good, leash, small, non, darti, pet]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, ...",0.8
157834,this is a well designed leash that offers the ...,5,"quality build, smooth retraction, great leash","quality build, smooth retraction, great leash","[qualiti, build, smooth, retract, great, leash]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19,...",1.0


## Upload Preprocess data to S3 and save locally

In [37]:
from sklearn.model_selection import train_test_split
import numpy as np

train_data_x, X_test, train_data_y, y_test = train_test_split(df['W2I'], df['overall_'], test_size=0.33, random_state=42)


In [46]:
train_data_x

26897     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
149426    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
100956    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
45653     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
55633     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
120112    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
103888    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
132193    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
147158    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
122196    [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 133, 1588, 755,...
Name: W2I, Length: 105545, dtype: object

In [130]:
X_test

119568    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
58145     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
47100     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
67350     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
120249    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
68937     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
98025     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23,...
74805     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10594     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
77800     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: W2I, Length: 51986, dtype: object

In [131]:
train_data_y

26897     1.0
149426    1.0
100956    1.0
45653     0.8
55633     0.8
         ... 
120112    0.8
103888    1.0
132193    0.8
147158    1.0
122196    0.8
Name: overall_, Length: 105545, dtype: float64

In [494]:
# import sagemaker
# from numpy import asarray

# session = sagemaker.Session()

# s3_prefix = 'train'

# data_dir = './data/pytorch'

# df.to_csv('./data/pytorch/train.csv', index=False)
# X_train.to_csv('./data/pytorch/train_x.csv')
# y_train.to_csv('./data/pytorch/train_y.csv')
# X_test.to_csv('../data/pytorch/test_x.csv')
# y_test.to_csv('../data/pytorch/test_y.csv')


# savetxt('../data/pytorch/train_x.csv', train_data_x, delimiter=',')
# savetxt('../data/pytorch/train_y.csv', train_data_y, delimiter=',')
# savetxt('../data/pytorch/test_x.csv', test_data_x, delimiter=',')
# savetxt('../data/pytorch/test_y.csv', test_data_y, delimiter=',')

# input_data = session.upload_data(path=data_dir, bucket='sagemaker-capstone-project', key_prefix=s3_prefix)

In [134]:
# with open(os.path.join('./data/pytorch', 'train_x.pkl'), "wb") as f:
#     pickle.dump(X_train, f)
# X_train.to_csv(os.path.join('./data/pytorch', 'train_x.csv'), header=False, index=False)
    
# with open(os.path.join('./data/pytorch', 'train_y.pkl'), "wb") as f:
#     pickle.dump(y_train, f)
# y_train.to_csv(os.path.join('./data/pytorch', 'train_y.csv'), header=False, index=False)
    

In [290]:
# import pandas as pd
# import os

# train_data_x = pd.read_pickle(os.path.join('./data/pytorch', "train_x.pkl"))
# train_data_y = pd.read_pickle(os.path.join('./data/pytorch', "train_y.pkl"))

# train_data_x = pd.read_csv(os.path.join('./data/pytorch', "train_x.csv"), header=None, names=None)
# train_data_y = pd.read_csv(os.path.join('./data/pytorch', "train_y.csv"),  header=None, names=None)

In [47]:
train_data_x.values

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 285, 525, 50]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 181, 4473, 48]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 90]),
       ...,
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 239, 0, 115, 1231]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 2, 0, 5]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 133, 1588, 755, 1961, 18, 1909, 0, 80, 168, 5])],
      dtype=object)

In [38]:
import torch.utils.data

torch_x_train = torch.from_numpy(np.array(train_data_x.values.tolist(), dtype=np.int16))
torch_y_train = torch.from_numpy(np.array(train_data_y.values.tolist(), dtype=np.float))

torch_x_test = torch.from_numpy(np.array(X_test.values.tolist(), dtype=np.int16))
torch_y_test = torch.from_numpy(np.array(y_test.values.tolist(), dtype=np.float))

In [39]:
print(torch_x_train.shape)
print(torch_y_train.shape)

torch.Size([105545, 20])
torch.Size([105545])


In [40]:
train_sample_ds = torch.utils.data.TensorDataset(torch_x_train, torch_y_train)
test_sample_ds = torch.utils.data.TensorDataset(torch_x_test, torch_y_test)

train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)
test_loader = torch.utils.data.DataLoader(test_sample_ds, batch_size=50)

In [41]:
# obtain one batch of training data
dataiter = iter(train_sample_dl)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 20])
Sample input: 
 tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,   11,  285,  525,   50],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    6,  181, 4473,   48],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    2,   90],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,   15,   28,   67,  583],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    7,    2,  216,   64],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,  887,  595,  190,  805,  542,  334, 1496],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

# Build PyTorch model

In [42]:
import torch.nn as nn

class LSTMClassifier(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        
        super(LSTMClassifier, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.dense = nn.Linear(hidden_dim, output_size)
        self.activation = nn.LeakyReLU()

    def forward(self, x):
        embeds = self.embedding(x.long())
        lstm_out, _ = self.lstm(embeds)
         
        out = self.dropout(lstm_out)
#         print(out.shape)
        
        out = self.dense(out)
#         print(out.shape)
        
        activation_out = self.activation(out)
#         softmax_out = softmax_out.view(50, -1, output_size)
#         activation_out = activation_out[:, -1] # get last batch of labels
#         print(softmax_out)
        return activation_out

In [43]:
class GRUNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        embeds = self.embedding(x.long())
        out, _= self.gru(embeds)
        out = self.dropout(out)
        out = self.fc(out)
        activation_out = self.relu(out)
        return activation_out[:, -1]

In [44]:
len(train_sample_dl)

2111

In [45]:
import time

clip=5 # gradient clipping
counter = 0


def train(model, train_loader, epochs, optimizer, loss_fn, device):
    epoch_times = []
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        start_time = time.clock()
        avg_loss = 0.
        counter = 0
        for batch_X, batch_y  in train_loader:  
            counter += 1
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
#             model.zero_grad() # Starting each batch, we detach the hidden state from how it was previously produced.
            optimizer.zero_grad()
    
            f = model.forward(batch_X)
            loss = loss_fn(f, batch_y.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
#             print(loss)
#             print(loss.item())
            avg_loss += loss.data.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        
        current_time = time.clock()
        
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, epochs, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
        
        print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
#         torch.save(model.state_dict(), './state_dict.pt')
    print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader)))
        
    return model

In [46]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [47]:
import torch.optim as optim

vocab_size = len(word2idx)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
in_epocs = 3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lstm_model = LSTMClassifier(vocab_size, output_size, embedding_dim, hidden_dim, n_layers).to(device)

print(lstm_model)

optimizer = optim.Adam(lstm_model.parameters(), 0.001)
loss_fn = torch.nn.MSELoss()

LSTMClassifier(
  (embedding): Embedding(7264, 400, padding_idx=0)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (dense): Linear(in_features=256, out_features=1, bias=True)
  (activation): LeakyReLU(negative_slope=0.01)
)


In [48]:
lstm_model = train(lstm_model, train_sample_dl, in_epocs, optimizer, loss_fn, device)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1......Step: 200/2111....... Average Loss for Epoch: 0.09881627066060901
Epoch 1......Step: 400/2111....... Average Loss for Epoch: 0.0797984169377014
Epoch 1......Step: 600/2111....... Average Loss for Epoch: 0.07329722442974647
Epoch 1......Step: 800/2111....... Average Loss for Epoch: 0.06934167568804696
Epoch 1......Step: 1000/2111....... Average Loss for Epoch: 0.06719747389107943
Epoch 1......Step: 1200/2111....... Average Loss for Epoch: 0.06552978387102484
Epoch 1......Step: 1400/2111....... Average Loss for Epoch: 0.06452119627435292
Epoch 1......Step: 1600/2111....... Average Loss for Epoch: 0.06371485159499571
Epoch 1......Step: 1800/2111....... Average Loss for Epoch: 0.06293680767839153
Epoch 1......Step: 2000/2111....... Average Loss for Epoch: 0.06252600147109479


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([45])) that is different to the input size (torch.Size([45, 20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/3 Done, Total Loss: 0.06217773140715598
Total Time Elapsed: 37.859999999999985 seconds
Total Training Time: 37.859999999999985 seconds
Epoch 2......Step: 200/2111....... Average Loss for Epoch: 0.05673592926003039
Epoch 2......Step: 400/2111....... Average Loss for Epoch: 0.057097256216220554
Epoch 2......Step: 600/2111....... Average Loss for Epoch: 0.0575214586344858
Epoch 2......Step: 800/2111....... Average Loss for Epoch: 0.05722232834203169
Epoch 2......Step: 1000/2111....... Average Loss for Epoch: 0.05727312531881034
Epoch 2......Step: 1200/2111....... Average Loss for Epoch: 0.05708364626237502
Epoch 2......Step: 1400/2111....... Average Loss for Epoch: 0.05715849434158632
Epoch 2......Step: 1600/2111....... Average Loss for Epoch: 0.05715720699517988
Epoch 2......Step: 1800/2111....... Average Loss for Epoch: 0.05701097245535089
Epoch 2......Step: 2000/2111....... Average Loss for Epoch: 0.0571044690143317
Epoch 2/3 Done, Total Loss: 0.05699185032551038
Total Time Ela

In [49]:
gru_model = GRUNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers).to(device)

print(gru_model)

GRUNet(
  (embedding): Embedding(7264, 400, padding_idx=0)
  (gru): GRU(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
)


In [50]:
gru_model = train(gru_model, train_sample_dl, in_epocs, optimizer, loss_fn, device)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1......Step: 200/2111....... Average Loss for Epoch: 0.6266861668229103
Epoch 1......Step: 400/2111....... Average Loss for Epoch: 0.6282054242491723
Epoch 1......Step: 600/2111....... Average Loss for Epoch: 0.6270927435656388
Epoch 1......Step: 800/2111....... Average Loss for Epoch: 0.6273189376667142
Epoch 1......Step: 1000/2111....... Average Loss for Epoch: 0.6263362368941308
Epoch 1......Step: 1200/2111....... Average Loss for Epoch: 0.626381298204263
Epoch 1......Step: 1400/2111....... Average Loss for Epoch: 0.6258667463915689
Epoch 1......Step: 1600/2111....... Average Loss for Epoch: 0.6257400956004858
Epoch 1......Step: 1800/2111....... Average Loss for Epoch: 0.6260462725162506
Epoch 1......Step: 2000/2111....... Average Loss for Epoch: 0.62540500728786


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([45])) that is different to the input size (torch.Size([45, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/3 Done, Total Loss: 0.6252916760068548
Total Time Elapsed: 34.75 seconds
Total Training Time: 34.75 seconds
Epoch 2......Step: 200/2111....... Average Loss for Epoch: 0.6269868022203445
Epoch 2......Step: 400/2111....... Average Loss for Epoch: 0.6279549877345562
Epoch 2......Step: 600/2111....... Average Loss for Epoch: 0.6256985849142075
Epoch 2......Step: 800/2111....... Average Loss for Epoch: 0.6265788702666759
Epoch 2......Step: 1000/2111....... Average Loss for Epoch: 0.6255598131418229
Epoch 2......Step: 1200/2111....... Average Loss for Epoch: 0.6259095249076684
Epoch 2......Step: 1400/2111....... Average Loss for Epoch: 0.6255333759954997
Epoch 2......Step: 1600/2111....... Average Loss for Epoch: 0.6253826293721795
Epoch 2......Step: 1800/2111....... Average Loss for Epoch: 0.6257050559255812
Epoch 2......Step: 2000/2111....... Average Loss for Epoch: 0.6249732454121113
Epoch 2/3 Done, Total Loss: 0.6251425569738499
Total Time Elapsed: 34.72000000000003 seconds
Total

In [ ]:
# %mkdir ../model

In [ ]:
# torch.save(lstm_model.state_dict(), "../model/lstm.dict")

## Test model

In [56]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

metrics = {
    'MSE': "Mean Squared Error",
    'RMSE': "Root Mean Squared Error",
    'R2': "R squared",
    'MAE': "Mean Absolute Error",
    'MAPE': "Mean Absolute Percentage Error",
    'SMAPE': "Symmetric Mean Absolute Percentage Error",
}

def _mean_absolute_percentage_error(y_true, y_pred):
    tr = y_true
    pr = y_pred
    return 100 * np.mean(abs((tr - pr) / tr))

def _symmetrical_mean_absolute_percentage_error(y_true, y_pred):
    tr = y_true
    pr = y_pred
    return 100 * np.mean((abs(tr - pr)) / ((abs(tr) + abs(pr)) / 2))

def compute_metrics(test_values, pred_values, metrics):
    k_metrics = dict.fromkeys(metrics)
    for met in list(metrics.keys()):
        if met == "MSE":
            k_metrics[met] = np.round(mean_squared_error(test_values, pred_values), 2)
        if met == "RMSE":
            k_metrics[met] = np.round(np.sqrt(mean_squared_error(test_values, pred_values)), 2)
        if met == "R2":
            k_metrics[met] = np.round(r2_score(test_values, pred_values), 2)
        if met == "MAE":
            k_metrics[met] = np.round(mean_absolute_error(test_values, pred_values), 2)
        if met == "MAPE":
            k_metrics[met] = np.round(_mean_absolute_percentage_error(test_values, pred_values), 2)
        if met == "SMAPE":
            k_metrics[met] = np.round(_symmetrical_mean_absolute_percentage_error(test_values, pred_values), 2)
    return k_metrics

In [57]:
def evaluate(model, test_x, test_y):
    targets = []
    start_time = time.clock()
    model.eval()

    outputs = model(test_x.to(device))
        
    print("Evaluation Time: {}".format(str(time.clock() - start_time)))
    
    sMAPE = 0
    mape = 0
    
    for i in range(len(outputs)):
        sMAPE += _symmetrical_mean_absolute_percentage_error(test_y[i], outputs[i].cpu().detach().numpy())
        mape += _mean_absolute_percentage_error(test_y[i],  outputs[i].cpu().detach().numpy())
     
    print("sMAPE: {}%".format(sMAPE))
    print("mape: {}%".format(mape))
    
    return sMAPE, mape

In [61]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [62]:
evaluate(lstm_model, torch_x_test, y_test.values)

Evaluation Time: 0.03000000000002956
sMAPE: 1346558.6801499128%
mape: 2007240.407519415%


(1346558.6801499128, 2007240.407519415)

In [63]:
evaluate(gru_model, torch_x_test.to(device), y_test.values)

Evaluation Time: 0.0
sMAPE: 8389318.453483287%
mape: 4583875.390186906%


(8389318.453483287, 4583875.390186906)

## Deploy on Sagemaker not finish

In [221]:
# import sagemaker

# session = sagemaker.Session()

# s3_prefix = 'train'

# data_dir = '../data/pytorch'

# input_data = session.upload_data(path=data_dir, bucket='sagemaker-capstone-project', key_prefix=s3_prefix)

In [162]:
# import sagemaker

# from sagemaker.pytorch import PyTorch

# role = sagemaker.get_execution_role()

# estimator = PyTorch(entry_point="train.py",
#                     source_dir="train",
#                     role=role,
#                     framework_version='0.4.0',
#                     py_version='py3',
#                     train_instance_count=1,
#                     train_instance_type='ml.p2.xlarge',
#                     hyperparameters={
#                         'epochs': 1,
#                         'hidden_dim': 256,
#                         'output_size' : 1
#                     })

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [163]:
# estimator.fit({'training': 's3://sagemaker-capstone-project/train/'})

2021-05-02 08:12:48 Starting - Starting the training job...
2021-05-02 08:12:51 Starting - Launching requested ML instancesProfilerReport-1619943168: InProgress
......
2021-05-02 08:14:19 Starting - Preparing the instances for training............
2021-05-02 08:16:20 Downloading - Downloading input data...
2021-05-02 08:16:39 Training - Downloading the training image...
2021-05-02 08:17:15 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-02 08:17:08,709 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-05-02 08:17:08,734 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-02 08:17:11,774 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-02 08:17:12,050 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generati

UnexpectedStatusException: Error for Training job sagemaker-pytorch-2021-05-02-08-12-47-958: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m train --epochs 1 --hidden_dim 256 --output_size 1"
Traceback (most recent call last):
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/ml/code/train.py", line 136, in <module>
    train_loader = _get_train_data_loader(args.batch_size, args.data_dir)
  File "/opt/ml/code/train.py", line 59, in _get_train_data_loader
    torch_x_train = torch.from_numpy(np.array(train_data_x.values.tolist(), dtype=np.int16))
ValueError: invalid literal for int() with base 10: '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 285, 525, 50]'

In [ ]:
# pytorch_estimator = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

# def predict(data, rows=512):
#     split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
#     predictions = np.array([])
#     for array in split_array:
#         predictions = np.append(predictions, pytorch_estimator.predict(array))
    
#     return predictions

In [ ]:
# predictions = predict(test_X.values)
# predictions = [round(num) for num in predictions]

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy_score(test_y, predictions)

In [ ]:
# estimator.delete_endpoint()